In [ ]:
import numpy as np
import zarr
from datetime import datetime
import os

import sys

if not sys.warnoptions:
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

Set path to store

In [ ]:
store_path = "s5p.zarr"

Set time extent with origin 01.04.2018

In [ ]:
now = datetime.now()
now_np = np.datetime64(now).astype('datetime64[D]')
now_np = np.datetime64("2025-08-12")
origin = np.datetime64("2018-04-01")
timesteps = int((now_np-origin).astype(int))

Set spatial and temporal extent

In [ ]:
x_extent = np.arange(4500000, 5400000, 10000)
y_extent = np.arange(1800000, 1200000, -10000)
time_extent = np.arange(0,timesteps,1)

Set shape of groups and chunk and shard shape

In [ ]:
shape = (time_extent.shape[0],y_extent.shape[0],x_extent.shape[0])

chunk_shape = (30,60,90)
shard_shape = (30,60,90)

x_shape = x_extent.shape
y_shape = y_extent.shape
time_shape = time_extent.shape

Load store if already created

In [ ]:
store = zarr.storage.LocalStore("s5p.zarr")
group = zarr.group(store=store)

Create store if not already created

In [ ]:
overwrite=False
store = zarr.storage.LocalStore(store_path)
group = zarr.create_group(store=store, overwrite=overwrite,
                        attributes={"Conventions": "CF-1.7",
                                    "freq": "1D",
                                    "platform": "S5P",
                                    "sensor": "TROPOMI",
                                    "spatial_resolution": 10000,
                                    "collection": "SENTINEL5P_10000m",
                                    "AREA_OR_POINT": "Area",
                                    "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

**ADD CO GROUP**

In [ ]:
CO = group.create_group(name="CO", overwrite=overwrite, 
                       attributes={"Conventions": "CF-1.7",
                                   "title": "TROPOMI/S5P CO Column 1-Orbit L2",
                                   "freq": "1D",
                                   "platform": "S5P",
                                   "sensor": "TROPOMI",
                                   "spatial_resolution": 10000,
                                   "collection": "SENTINEL5P_10000m",
                                   "AREA_OR_POINT": "Area",
                                   "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

CO.create_array(name="carbon_monoxide_total_column",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0001},
                overwrite=overwrite)

CO.create_array(name="carbon_monoxide_total_column_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0001},
                overwrite=overwrite)

CO.create_array(name="carbon_monoxide_total_column_corrected",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0001},
                overwrite=overwrite)

CO.create_array(name="qa_value",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,
                            "scale_factor": 0.1},
                overwrite=overwrite)

x_array = CO.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = CO.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = CO.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2018-04-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

zarr.consolidate_metadata(os.path.join(store_path, "CO"))

**ADD SO2 GROUP**

In [ ]:
SO2 = group.create_group(name="SO2", overwrite=overwrite, 
                       attributes={"Conventions": "CF-1.7",
                                   "title": "TROPOMI/S5P Sulphur Dioxide SO2",
                                   "freq": "1D",
                                   "platform": "S5P",
                                   "sensor": "TROPOMI",
                                   "spatial_resolution": 10000,
                                   "collection": "SENTINEL5P_10000m",
                                   "AREA_OR_POINT": "Area",
                                   "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

SO2.create_array(name="sulphur_dioxide_total_vertical_column",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.000001},
                overwrite=overwrite)

SO2.create_array(name="sulphur_dioxide_total_vertical_column_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.000001},
                overwrite=overwrite)

SO2.create_array(name="qa_value",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,
                            "scale_factor": 0.1},
                overwrite=overwrite)

x_array = SO2.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = SO2.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = SO2.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2018-04-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

zarr.consolidate_metadata(os.path.join(store_path, "SO2"))

**ADD AER GROUP**

In [ ]:
AER = group.create_group(name="AER_AI", overwrite=overwrite, 
                       attributes={"Conventions": "CF-1.7",
                                   "title": "TROPOMI/S5P Aerosol Index",
                                   "freq": "1D",
                                   "platform": "S5P",
                                   "sensor": "TROPOMI",
                                   "spatial_resolution": 10000,
                                   "collection": "SENTINEL5P_10000m",
                                   "AREA_OR_POINT": "Area",
                                   "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

AER.create_array(name="aerosol_index_354_388",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="aerosol_index_340_388",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="aerosol_index_335_367",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="aerosol_index_354_388_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="aerosol_index_340_388_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="aerosol_index_335_367_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.001},
                overwrite=overwrite)

AER.create_array(name="qa_value",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,
                            "scale_factor": 0.1},
                overwrite=overwrite)

x_array = AER.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = AER.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = AER.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2018-04-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

zarr.consolidate_metadata(os.path.join(store_path, "AER_AI"))

**ADD HCHO GROUP**

In [ ]:
HCHO = group.create_group(name="HCHO", overwrite=overwrite, 
                       attributes={"Conventions": "CF-1.7",
                                   "title": "TROPOMI/S5P Formaldehyde HCHO",
                                   "freq": "1D",
                                   "platform": "S5P",
                                   "sensor": "TROPOMI",
                                   "spatial_resolution": 10000,
                                   "collection": "SENTINEL5P_10000m",
                                   "AREA_OR_POINT": "Area",
                                   "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

HCHO.create_array(name="formaldehyde_tropospheric_vertical_column",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.000001},
                overwrite=overwrite)

HCHO.create_array(name="formaldehyde_tropospheric_vertical_column_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.000001},
                overwrite=overwrite)

HCHO.create_array(name="qa_value",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,
                            "scale_factor": 0.1},
                overwrite=overwrite)

x_array = HCHO.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = HCHO.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = HCHO.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2018-04-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

zarr.consolidate_metadata(os.path.join(store_path, "HCHO"))

**ADD NO2 GROUP**

In [ ]:
NO2 = root.create_group(name="NO2", overwrite=overwrite, 
                       attributes={"Conventions": "CF-1.7",
                                   "title": "TROPOMI/S5P Nitrogen Dioxide NO2",
                                   "freq": "1D",
                                   "platform": "S5P",
                                   "sensor": "TROPOMI",
                                   "spatial_resolution": 10000,
                                   "collection": "SENTINEL5P_10000m",
                                   "AREA_OR_POINT": "Area",
                                   "crs_wkt": "PROJCS[\"WGS84/Equi7Europe\",GEOGCS[\"WGS84\",DATUM[\"WGS_1984\",SPHEROID[\"WGS84\",6378137,298.257223563,AUTHORITY[\"EPSG\",\"7030\"]],AUTHORITY[\"EPSG\",\"6326\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4326\"]],PROJECTION[\"Azimuthal_Equidistant\"],PARAMETER[\"latitude_of_center\",53],PARAMETER[\"longitude_of_center\",24],PARAMETER[\"false_easting\",5837287.82],PARAMETER[\"false_northing\",2121415.696],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"27704\"]]"})

NO2.create_array(name="nitrogendioxide_tropospheric_column",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0000001},
                overwrite=overwrite)

NO2.create_array(name="nitrogendioxide_tropospheric_column_precision",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0000001},
                overwrite=overwrite)

NO2.create_array(name="nitrogendioxide_tropospheric_column_precision_kernel",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0000001},
                overwrite=overwrite)

NO2.create_array(name="averaging_kernel",
                shape=(time_extent.shape[0],y_extent.shape[0],x_extent.shape[0], 34),
                chunks=(30,60,90,34),
                shards=(30,60,90,34),
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x", "layer"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.0001},
                overwrite=overwrite)

NO2.create_array(name="air_mass_factor_troposphere",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.01},
                overwrite=overwrite)

NO2.create_array(name="air_mass_factor_total",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.01},
                overwrite=overwrite)

NO2.create_array(name="tm5_tropopause_layer_index",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,},
                overwrite=overwrite)

NO2.create_array(name="tm5_constant_a",
                shape=(34,2),
                chunks=(34,2),
                shards=(34,2),
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["layer", "vertices"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.1},
                overwrite=overwrite)

NO2.create_array(name="tm5_constant_b",
                shape=(34,2),
                chunks=(34,2),
                shards=(34,2),
                compressors = zarr.codecs.BloscCodec(),
                dtype="int16",
                fill_value=-9999,
                dimension_names=["layer", "vertices"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -9999,
                            "scale_factor": 0.1},
                overwrite=overwrite)

NO2.create_array(name="qa_value",
                shape=shape,
                chunks=chunk_shape,
                shards=shard_shape,
                compressors = zarr.codecs.BloscCodec(),
                dtype="int8",
                fill_value=-99,
                dimension_names=["time", "y", "x"],
                config={"write_empty_chunks":False},
                attributes={"_FillValue": -99,
                            "scale_factor": 0.1},
                overwrite=overwrite)

x_array = NO2.create_array(name="x",
            shape=x_shape,
            chunks=x_shape,
            dtype="float64",
            dimension_names=["x"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "X",
                        "long_name": "x coordinate of projection",
                        "standard_name": "projection_x_coordinate",
                        "units": "m"},
            overwrite=overwrite)

y_array = NO2.create_array(name="y",
            shape=y_shape,
            chunks=y_shape,
            dtype="float64",
            dimension_names=["y"],
            attributes={"_FillValue": "AAAAAAAA+H8=", #fill value is NaN
                        "axis": "Y",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

time_array = NO2.create_array(name="time",
            shape=time_shape,
            dtype="int64",
            dimension_names=["time"],
            attributes={"units": "days since 2018-04-01",
                        "calendar": "proleptic_gregorian",
                        "long_name": "date of acquisition",
                        "standard_name": "time"},
            overwrite=overwrite)

layer = NO2.create_array(name="layer",
            shape=(34),
            chunks=(34),
            dtype="float32",
            dimension_names=["layer"],
            attributes={"axis": "layer",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

vertices = NO2.create_array(name="vertices",
            shape=(2),
            chunks=(2),
            dtype="float32",
            dimension_names=["vertices"],
            attributes={"axis": "vertices",
                        "long_name": "y coordinate of projection",
                        "standard_name": "projection_y_coordinate",
                        "units": "m"},
            overwrite=overwrite)

layer[:] = [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
       28., 29., 30., 31., 32., 33.]

vertices[:] = [0,1]

x_array[:] = x_extent
y_array[:] = y_extent
time_array[:] = time_extent

zarr.consolidate_metadata(os.path.join(store_path, "NO2"))

In [ ]:
zarr.consolidate_metadata(store)